In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np

# 将上一级目录添加到模块搜索路径
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [2]:
# 处理元数据
import pandas as pd

# 读取 CSV 文件
csv_file = "../data/papers_metadata.csv" 
metadata_df = pd.read_csv(csv_file)

# 打印数据表格
print(metadata_df.columns.to_list())
# CSV标题：paper_title,author_ids,author_names,venue,research_area,keywords,tldr,abstract,url,pdf_url,attachment_url,pdf_path

# use a for loop to print first 2 row all data columns
for index, row in metadata_df[:2].iterrows():
     print(row["paper_title"])
     print(row["abstract"])
     print(row["author_names"].split(","))
     print(row["author_ids"].split(","))
     print(row["pdf_path"])
     print(row["tldr"])
     print(row["venue"])
     print(row["research_area"])
     print(row["keywords"].split(","))
     print(row["url"])
     print("-" * 80)


['paper_title', 'author_ids', 'author_names', 'venue', 'research_area', 'keywords', 'tldr', 'abstract', 'url', 'pdf_url', 'attachment_url', 'pdf_path']
Stress-Testing Capability Elicitation With Password-Locked Models
To determine the safety of large language models (LLMs), AI developers must be able to assess their dangerous capabilities. But simple prompting strategies often fail to elicit an LLM’s full capabilities. One way to elicit capabilities more robustly is to fine-tune the LLM to complete the task. In this paper, we investigate the conditions under which fine-tuning-based elicitation suffices to elicit capabilities. To do this, we introduce password-locked models, LLMs fine-tuned such that some of their capabilities are deliberately hidden. Specifically, these LLMs are trained to exhibit these capabilities only when a password is present in the prompt, and to imitate a much weaker LLM otherwise. Password-locked models enable a novel method of evaluating capabilities elicitati

In [4]:
import re

def parse_title_to_filename(title):
    # Remove any non-alphanumeric characters and replace spaces with underscores
    filename = re.sub(r'\W+', '_', title)
    return filename + '.pdf'

# Test the function with an example
example_title = "Stress-Testing Capability Elicitation With Password-Locked Models"
pdf_filename = parse_title_to_filename(example_title)
print(pdf_filename)

Stress_Testing_Capability_Elicitation_With_Password_Locked_Models.pdf


In [6]:
for index, row in metadata_df[:2].iterrows():
     pdf_filename = parse_title_to_filename(row["paper_title"])
     print(pdf_filename)

Stress_Testing_Capability_Elicitation_With_Password_Locked_Models.pdf
Cooperative_Hardware_Prompt_Learning_for_Snapshot_Compressive_Imaging.pdf


In [53]:
from llm.ollama_model import OllamaModel
llm = OllamaModel()
response = llm.generate_response(prompt="What is the impact of llm?")
print(response)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:llm.ollama_model:Response from Ollama model: model='deepseek-r1:7b' created_at='2025-02-21T16:54:38.053241Z' done=True done_reason='stop' total_duration=3681149125 load_duration=563057209 prompt_eval_count=11 prompt_eval_duration=2234000000 eval_count=36 eval_duration=881000000 message=Message(role='assistant', content="<think>\n\n</think>\n\nDeepSeek is a Chinese company dedicated to making AGI a reality. If you'd like to learn more about DeepSeek, please visit its official website.", images=None, tool_calls=None)


<think>

</think>

DeepSeek is a Chinese company dedicated to making AGI a reality. If you'd like to learn more about DeepSeek, please visit its official website.


In [54]:
from llm.ollama_model import OllamaModel
from paper_agent import PDFAnalyzer

# create ollama model
llm = OllamaModel()

for index, row in metadata_df[:1].iterrows():
    pdf_path = "../data/pdfs/neurips/2024/Stress_Testing_Capability_Elicitation_With_Password_Locked_Models.pdf"
    
    # Create a PDFAnalyzer object
    # CSV标题：paper_title,author_ids,author_names,venue,research_area,keywords,tldr,abstract,url,pdf_url,attachment_url,pdf_path

    pdf_analyzer = PDFAnalyzer(
        title= row["paper_title"],
        abstract= row["abstract"],
        author_names= row["author_names"].split(","),
        author_ids= row["author_ids"].split(","),
        pdf_path=pdf_path,
        tldr=row["tldr"],
        keywords=row["keywords"].split(","),
        llm=llm
    )

In [55]:
pdf_analyzer.get_title()

'Stress-Testing Capability Elicitation With Password-Locked Models'

In [ ]:
# pdf_analyzer.parse_all_in_one()
text = pdf_analyzer.extract_text(pdf_path=pdf_analyzer.get_pdf_path())
text_lines =pdf_analyzer.extract_text_lines(text)
print(len(text_lines))

/Users/jiwenyu/Dev/jwgen/jwenv/lib/python3.11/site-packages/pdfminer/psparser.py:298: ResourceWarning: unclosed <socket.socket fd=75, family=2, type=1, proto=6, laddr=('127.0.0.1', 55929), raddr=('127.0.0.1', 11434)>
  self._parse1 = self._parse_literal
/Users/jiwenyu/Dev/jwgen/jwenv/lib/python3.11/site-packages/pdfminer/psparser.py:298: ResourceWarning: unclosed <socket.socket fd=88, family=2, type=1, proto=6, laddr=('127.0.0.1', 49543), raddr=('127.0.0.1', 11434)>
  self._parse1 = self._parse_literal
INFO:paper_agent:Converted PDF to text: Stress-Testing Capability Elicitation With
Password-Locked Models

Ryan Greenblatt∗
Redwood Research



1402


In [57]:
title_indices = pdf_analyzer.get_section_title_indices(text_lines)
print(title_indices)

INFO:paper_agent:Detected title: limitations at line 1120


{'abstract': 14, 'introduction': 36, 'related_work': 145, 'experiment': 182, 'methodology': 388, 'limitations': 468, 'conclusion': 509, 'references': 522, 'limitations_0': 1120}


In [58]:
section_lines_dict = pdf_analyzer.split_text_lines_by_section_title(text_lines, title_indices)
print(section_lines_dict.keys())
print(section_lines_dict.get("abc", None))
print(section_lines_dict["introduction"])

dict_keys(['abstract', 'introduction', 'related_work', 'experiment', 'methodology', 'limitations', 'conclusion', 'references', 'limitations_0'])
None
['Introduction', 'To make training and deployment decisions, large language model (LLM) developers rely on mea-', 'suring dangerous capabilities of the systems they create (Anthropic, 2023; OpenAI, 2023). Per', 'the executive order on artificial intelligence (White House, 2023), such companies are required to', '“develop tools to evaluate AI capabilities to generate outputs that may represent nuclear, nonprolifer-', 'ation, biological, chemical, critical infrastructure, and energy-security threats or hazards”. Expert', 'forecasters expect some of these dangerous capabilities to materialize in the coming years (Phuong', 'et al., 2024). Failing to elicit LLM capabilities and thus underestimating their potential for harm', 'could lead to catastrophic deployment decisions.1', '*Equal contribution. Author contributions are listed in Appendix A

In [59]:
section_lines_dict = pdf_analyzer.split_text_lines_by_section_title(text_lines, title_indices)
print(section_lines_dict.keys())
print(section_lines_dict.get("abstract", None)[:2])

dict_keys(['abstract', 'introduction', 'related_work', 'experiment', 'methodology', 'limitations', 'conclusion', 'references', 'limitations_0'])
['Abstract', 'To determine the safety of large language models (LLMs), AI developers must']


In [60]:
abstract = pdf_analyzer.extract_abstract_text_from_paper(section_lines_dict)
print(abstract)

Abstract
To determine the safety of large language models (LLMs), AI developers must
be able to assess their dangerous capabilities. But simple prompting strategies
often fail to elicit an LLM’s full capabilities. One way to elicit capabilities more
robustly is to fine-tune the LLM to complete the task. In this paper, we inves-
tigate the conditions under which fine-tuning-based elicitation suffices to elicit
capabilities. To do this, we introduce password-locked models, LLMs fine-tuned
such that some of their capabilities are deliberately hidden. Specifically, these
LLMs are trained to exhibit these capabilities only when a password is present
in the prompt, and to imitate a much weaker LLM otherwise. Password-locked
models enable a novel method of evaluating capabilities elicitation methods, by
testing whether these password-locked capabilities can be elicited without using
the password. We find that a few high-quality demonstrations are often sufficient
to fully elicit password-lock

In [71]:
# 构造原始的作者信息，根据作者的名字和 ID
author_names = row["author_names"].split(",")
author_ids = row["author_ids"].split(",")
authors = []
for author_name, author_id in zip(author_names, author_ids):
    author = {
        "name": author_name,
        "author_ids": author_id,
        "affiliation": None,
        "email": None,
        "contribution_order": 0,
        "is_corresponding": True,
        "nationality": ""
    }
    authors.append(author)
print(authors)

[{'name': "['Ryan Greenblatt'", 'author_ids': "['~Ryan_Greenblatt1'", 'affiliation': None, 'email': None, 'contribution_order': 0, 'is_corresponding': True, 'nationality': ''}, {'name': " 'Fabien Roger'", 'author_ids': " '~Fabien_Roger1'", 'affiliation': None, 'email': None, 'contribution_order': 0, 'is_corresponding': True, 'nationality': ''}, {'name': " 'Dmitrii Krasheninnikov'", 'author_ids': " '~Dmitrii_Krasheninnikov1'", 'affiliation': None, 'email': None, 'contribution_order': 0, 'is_corresponding': True, 'nationality': ''}, {'name': " 'David Krueger']", 'author_ids': " '~David_Krueger1']", 'affiliation': None, 'email': None, 'contribution_order': 0, 'is_corresponding': True, 'nationality': ''}]


In [92]:
import json

print(text_lines[:2])
print(title_indices)
print(pdf_analyzer.get_author_names())
print(pdf_analyzer.get_author_ids())

authors_augmented_info_dict = pdf_analyzer.extract_augmented_authors_info(llm, text_lines, 
                                                                          title_indices,
                                                                          pdf_analyzer.author_names,
                                                                          pdf_analyzer.author_ids)

print(json.dumps(authors_augmented_info_dict, indent=2))
        

INFO:paper_agent:Extracted author info lines: ['Stress-Testing Capability Elicitation With', 'Password-Locked Models', 'Ryan Greenblatt∗', 'Redwood Research', 'ryan@rdwrs.com', 'Fabien Roger∗', 'Redwood Research', 'fabien.d.roger@gmail.com', 'Dmitrii Krasheninnikov', 'University of Cambridge', 'dk655@cam.ac.uk', 'David Krueger', 'University of Cambridge', 'david.scott.krueger@gmail.com']


['Stress-Testing Capability Elicitation With', 'Password-Locked Models']
{'abstract': 14, 'introduction': 36, 'related_work': 145, 'experiment': 182, 'methodology': 388, 'limitations': 468, 'conclusion': 509, 'references': 522, 'limitations_0': 1120}
["['Ryan Greenblatt'", " 'Fabien Roger'", " 'Dmitrii Krasheninnikov'", " 'David Krueger']"]
["['~Ryan_Greenblatt1'", " '~Fabien_Roger1'", " '~Dmitrii_Krasheninnikov1'", " '~David_Krueger1']"]


INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:llm.ollama_model:Response from Ollama model: model='deepseek-r1:7b' created_at='2025-02-21T20:22:06.795004Z' done=True done_reason='stop' total_duration=27065484375 load_duration=28139375 prompt_eval_count=804 prompt_eval_duration=2696000000 eval_count=861 eval_duration=24340000000 message=Message(role='assistant', content='<think>\n好的，我现在需要处理用户的查询。用户给了一个包含论文文本和作者信息的JSON数组，并要求根据这些信息生成一个更新后的作者JSON数组。\n\n首先，我会仔细阅读用户的要求。论文文本中有作者姓名、机构和电子邮件等信息，而JSON数组中的部分字段缺失或有误。我的任务是清理作者姓名，补充full_name，并提取email和affiliation。此外，还需要处理一些细节，比如is_corresponding字段的设置以及nationality的推测。\n\n接下来，我分析输入的内容。论文文本中有一系列名字、机构名称、邮箱等信息，而JSON数组中有四位作者，但每个都有不同的问题：name包含引号和星号，author_id格式不一致，email为空，affiliation为空，is_corresponding设为false，nationality缺失。\n\n首先处理name字段。原始name中有引号和可能的星号，比如"Ryan Greenblatt*"。根据要求，需要去除这些多余符号，同时提取full_name，并判断是否是对应作者（is_corresponding）。对于带有*的情况，is_corresponding设为true。\n\n然后看email部分。有些作者的email为空，需要从论文文本中提取出来。比如，Redwood Researc

[
  {
    "name": "Ryan Greenblatt",
    "author_id": "~Ryan_Greenblatt1",
    "affiliation": [
      "Redwood Research"
    ],
    "email": "ryan@rdwrs.com",
    "contribution_order": 0,
    "is_corresponding": true,
    "nationality": "United States"
  },
  {
    "name": "Fabien Roger",
    "author_id": "~Fabien_Roger1",
    "affiliation": [
      "Redwood Research"
    ],
    "email": "fabien.d.roger@gmail.com",
    "contribution_order": 1,
    "is_corresponding": true,
    "nationality": "France"
  },
  {
    "name": "Dmitrii Krasheninnikov",
    "author_id": "~Dmitrii_Krasheninnikov1",
    "affiliation": [
      "Redwood Research"
    ],
    "email": "dk655@cam.ac.uk",
    "contribution_order": 2,
    "is_corresponding": false,
    "nationality": "United Kingdom"
  },
  {
    "name": "David Krueger",
    "author_id": "~David_Krueger1",
    "affiliation": [
      "Redwood Research"
    ],
    "email": "david.scott.krueger@gmail.com",
    "contribution_order": 3,
    "is_correspondi

In [109]:
conclusion_text = pdf_analyzer.extract_conclusion(section_lines_dict)
print(conclusion_text)



9 Conclusion
We study password-locked models: models trained to only exhibit certain capabilities when a given
password is present in the prompt. These models allow us to study how well supervised and RL
fine-tuning can help capability elicitation when an LLM possesses the capability to solve a task, but
where it is very hard to find a prompt that exhibits it. Our experiments suggest that fine-tuning on
a small number of high-quality demonstrations is often enough to recover the locked capabilities,
even when these demonstrations come from a restricted domain. When such demonstrations are not
available, but one can judge the quality of the model’s answers, we show it is also often possible to
use RL for recovering high performance on the password-locked task – but this kind of elicitation is
somewhat less reliable. While our password-locked models differ from capability elicitation failures
that might occur naturally, our study may guide future model evaluation efforts by providing a
m

In [121]:
print(section_lines_dict.keys())
print(section_lines_dict.get("references"))
print(len(section_lines_dict.get("references")))
print(section_lines_dict.get("references")[590:600])


dict_keys(['abstract', 'introduction', 'related_work', 'experiment', 'methodology', 'limitations', 'conclusion', 'references', 'limitations_0', 'instruction'])
['References', 'Anthony, T., Tian, Z., and Barber, D. Thinking fast and slow with deep learning and tree search.', 'Advances in neural information processing systems, 30, 2017. 8', 'Anthropic. Anthropics responsible scaling policy.', 'https://www.anthropic.com/index/', 'anthropics-responsible-scaling-policy, 2023. 1, 2, 4', 'Anthropic. Responsible scaling policy evaluations report – claude 3 opus. https://cdn.sanity.', 'io/files/4zrzovbb/website/210523b8e11b09c704c5e185fd362fe9e648d457.pdf,', '2024. 6', 'Anwar, U., Saparov, A., Rando, J., Paleka, D., Turpin, M., Hase, P., Lubana, E. S., Jenner, E., Casper,', 'S., Sourbut, O., et al. Foundational challenges in assuring alignment and safety of large language', 'models. arXiv preprint arXiv:2404.09932, 2024. 3', 'Austin, J., Odena, A., Nye, M., Bosma, M., Michalewski, H., Dohan, D.

In [124]:
references_list = pdf_analyzer.extract_formated_references_list(llm, section_lines_dict)

INFO:paper_agent:Split references into 23 chunks
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:llm.ollama_model:Response from Ollama model: model='deepseek-r1:7b' created_at='2025-02-21T21:12:44.205435Z' done=True done_reason='stop' total_duration=32675093042 load_duration=53916917 prompt_eval_count=1322 prompt_eval_duration=4630000000 eval_count=940 eval_duration=27990000000 message=Message(role='assistant', content='<think>\nAlright, I\'m trying to tackle this problem where I need to extract key citation information from given fragments. Let\'s break it down step by step.\n\nFirst, I see that there are multiple citation fragments provided in the input. My task is to parse each of these and determine if they\'re valid references. If they aren\'t, I should return an empty list. If they are, I need to extract specific fields like title, author, year, journal, and web_url.\n\nLooking at the first fragment: "Anthony, T., Tian, Z., and Barber, D. Thin

In [126]:
print(references_list)
print(json.dumps(references_list,indent=2))

[{'title': 'Thinking fast and slow with deep learning and tree search', 'author': 'Anthony, T., Tian, Z., and Barber, D.', 'year': 2017, 'journal': 'unknown', 'web_url': 'https://www.anthropic.com/index/anthropics-responsible-scaling-policy'}, {'title': 'Anthropics responsible scaling policy', 'author': 'unknown', 'year': 2023, 'journal': 'unknown', 'web_url': 'https://www.anthropic.com/index/anthropics-responsible-scaling-policy'}, {'title': 'Foundational challenges in assuring alignment and safety of large language models', 'author': 'Anwar, U., Saparov, A., Rando, J., Paleka, D., Turpin, M., Hase, P., Lubana, E., et al.', 'year': 2023, 'journal': 'unknown', 'web_url': 'https://arxiv.org/abs/2312.04678'}, {'title': 'Program synthesis with large language models', 'author': 'Austin, J., Odena, A., Nye, M., Bosma, M., Michalewski, H., and et al.', 'year': 2021, 'journal': 'arXiv preprint arXiv:2108.07732', 'web_url': 'https://arxiv.org/abs/2108.07732'}, {'title': 'Cooperative Hardware-P

In [128]:
embedding_list = pdf_analyzer.embed_main_text(llm, section_lines_dict)

INFO:paper_agent:Split text into 41 chunks
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:

In [129]:
print(embedding_list[:2])

[[0, 'abstract', 'Abstract\nTo determine the safety of large language models (LLMs), AI developers must\nbe able to assess their dangerous capabilities. But simple prompting strategies\noften fail to elicit an LLM’s full capabilities. One way to elicit capabilities more\nrobustly is to fine-tune the LLM to complete the task. In this paper, we inves-\ntigate the conditions under which fine-tuning-based elicitation suffices to elicit\ncapabilities. To do this, we introduce password-locked models, LLMs fine-tuned\nsuch that some of their capabilities are deliberately hidden. Specifically, these\nLLMs are trained to exhibit these capabilities only when a password is present\nin the prompt, and to imitate a much weaker LLM otherwise. Password-locked\nmodels enable a novel method of evaluating capabilities elicitation methods, by\ntesting whether these password-locked capabilities can be elicited without using\nthe password. We find that a few high-quality demonstrations are often sufficient

In [ ]:
# test parse all
pdf_analyzer.parse_all()